# Sequence to Sequence model to extract summary,objective from resume (input: resume line by line)


# Trained data 40 resumes and 10 testing resumes to validate

## Example 1:
###             Input:

###           Output:


In [24]:
from __future__ import print_function
import csv,os
import collections
from keras.models import Model,load_model
from keras.layers import Input,LSTM,Dense
from keras.callbacks import ModelCheckpoint
import numpy as np

batch_size=90
epochs=50
latent_dim=128
data_path="/home/santhosh/resumes_folder/custom_annotator/annotator-server/static/files/Data_Tracter_Resumes_in_TXT/csv_1"

## vectorizing the data

In [25]:
input_resumes=[]
output_summary=[]
input_tokens=collections.Counter()
output_tokens=set()
files=os.listdir(data_path)
count=0
line=0;
for file in files:
    with open(data_path+'/'+file,'r') as csv_file:
        reader=csv.reader(csv_file)
        count=0
        for Input_text,output in reader:
            Input_text=Input_text.strip().lower()
            Input_text=Input_text+' \n'
            if(output=='1'):
                output_text=Input_text+' \n'
            else:
                output_text=""
            
            # We use "<SOL>" as the "start sequence" character
            # for the targets, and "<EOL>" as "end sequence" character.
            output_text='<SOL> '+output_text+' <EOL>'
            input_resumes.append(Input_text)
            output_summary.append(output_text)
            for word in Input_text.split():
                if word not in input_tokens:
                    input_tokens[word]+=1
            for word in output_text.split():
                if word not in output_tokens:
                    output_tokens.add(word)
            if count==40:
                break
            count+=1


In [26]:
print(len(input_tokens))
num_encoder_tokens=min(len(input_tokens),3500)
input_tokens=[word for word,count in input_tokens.most_common(num_encoder_tokens-1)]
input_tokens=sorted(list(input_tokens))
output_tokens=sorted(list(output_tokens))
num_decoder_tokens=len(output_tokens)
max_encoder_seq_len=max([len(text.split()) for text in input_resumes])
max_decoder_seq_len=max([len(text.split()) for text in output_summary])

print('Number of samples:',len(input_resumes))
print('number of unique input token:',num_encoder_tokens)
print('number of unique output token:',num_decoder_tokens)
print('Max Sequence length for inputs:',max_encoder_seq_len)
print('Max Sequence length for outputs:',max_decoder_seq_len)


4221
Number of samples: 2043
number of unique input token: 3500
number of unique output token: 1568
Max Sequence length for inputs: 26
Max Sequence length for outputs: 24


## defining token2index

In [27]:
input_token2index=dict([(word,i) for i,word in enumerate(input_tokens)])
output_token2index=dict([(word,i) for i,word in enumerate(output_tokens)])
input_token2index['UNK']=num_encoder_tokens-1

## defing encoder_input,decoder_input and decoder_output

In [28]:
encoder_input_data=np.zeros((len(input_resumes),max_encoder_seq_len,num_encoder_tokens),dtype='float32')
decoder_input_data=np.zeros((len(input_resumes),max_decoder_seq_len,num_decoder_tokens),dtype='float32')
decoder_target_data=np.zeros((len(input_resumes),max_decoder_seq_len,num_decoder_tokens),dtype='float32')


## creating training dataset

In [29]:
for i,(input_text,target_text) in enumerate(zip(input_resumes,output_summary)):
    for t,word in enumerate(input_text.split()[:max_encoder_seq_len]):
        if word not in input_token2index:
            word="UNK"
        encoder_input_data[i,t,input_token2index[word]]=1
        
    for t,word in enumerate(target_text.split()):
        decoder_input_data[i,t,output_token2index[word]]=1
        # decoder_target_data is ahead of decoder_input_data by one timestep
        if t>0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i,t-1,output_token2index[word]]=1


## Define an input sequence and process it.

In [30]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]


## Set up the decoder, using encoder_states as initial state.

In [31]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

## Define the model that will turn
## `encoder_input_data` & `decoder_input_data` into `decoder_target_data`

In [32]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=["accuracy"])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, None, 3500)   0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, None, 1568)   0                                            
__________________________________________________________________________________________________
lstm_5 (LSTM)                   [(None, 128), (None, 1858048     input_7[0][0]                    
__________________________________________________________________________________________________
lstm_6 (LSTM)                   [(None, None, 128),  868864      input_8[0][0]                    
                                                                 lstm_5[0][1]                     
          

## function to test model

In [33]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_word_index = dict(
    (i, word) for word, i in input_token2index.items())
reverse_target_word_index = dict(
    (i, word) for word, i in output_token2index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, output_token2index['<SOL>']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = reverse_target_word_index[sampled_token_index]
        decoded_sentence += ' '+sampled_word

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_word == '<EOL>' or
           len(decoded_sentence.split()) > max_decoder_seq_len):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]
    if('<EOL>' in decoded_sentence):
        decoded_sentence=" ".join(decoded_sentence.split()[:-1])
    return decoded_sentence


## Run training

In [ ]:
iteration_file="/home/santhosh/resumes_folder/keras/extract_summary_and_objective/resume_line_level/iteration_resume_line_level.txt"
iteration=0

try:
    file=open(iteration_file,'r')
    last_line=file.read().split('\n')[-2]
    print('file_data,',last_line)
    iteration=int(last_line.split(':')[1])
    
    #print(iteration)
    file.close()
    # load weights
    print('loading the weights')
    file_path='/home/santhosh/resumes_folder/keras/extract_summary_and_objective/resume_line_level/model/resume_line_level_iteration_'+str(iteration)+'.h5'
    model=load_model(file_path)
    # estimate accuracy on whole dataset using loaded weights
    scores = model.evaluate([encoder_input_data, decoder_input_data], decoder_target_data,verbose=0)
    print("%s: %.2f%%\n\n" % (model.metrics_names[1], scores[1]*100))
    print("Testing Samples\n"+"-"*50)
    print("-"*50)
    index=int(np.random.randint(len(input_resumes)/40*0.8))
    test_input=""
    test_output=""
    for i in range(40):
        encoded_input_sequence=(encoder_input_data[index]).reshape((1,max_encoder_seq_len,num_encoder_tokens))
        #print(encoded_input_sequence.shape,max_encoder_seq_len,num_encoder_tokens,max_encoder_seq_len*num_encoder_tokens)
        #print(encoded_input_sequence)
        #.reshape((1,max_encoder_seq_len,num_encoder_tokens))
        output_sequence=decode_sequence(encoded_input_sequence)
        test_input+=input_resumes[index]
        test_output+=output_sequence
        index+=1
    print("-"*50)
    print(test_input)
    print("---OUTPUT-----")
    print(test_output)
    print(" "*50+"-"*50)

except:
    print('no file exist')

# checkpoint
filepath="/home/santhosh/resumes_folder/keras/extract_summary_and_objective/resume_line_level/checkpoints/resume_line_level_checkpoints.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

while True:
    try:
        print('Iteration:',iteration+1)
        #training
        model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
              batch_size=batch_size,
              epochs=epochs,
              validation_split=0.2,callbacks=callbacks_list)
        #prepare sample_data to test 5 samples:
        print("-"*50)
        index=int(np.random.randint(len(input_resumes)/40*0.8))
        test_input=""
        test_output=""
        for i in range(40):
            encoded_input_sequence=(encoder_input_data[index]).reshape((1,max_encoder_seq_len,num_encoder_tokens))
            #print(encoded_input_sequence.shape,max_encoder_seq_len,num_encoder_tokens,max_encoder_seq_len*num_encoder_tokens)
            #print(encoded_input_sequence)
            #.reshape((1,max_encoder_seq_len,num_encoder_tokens))
            output_sequence=decode_sequence(encoded_input_sequence)
            test_input+=input_resumes[index]
            test_output+=output_sequence
            index+=1
        print("-"*50)
        print(test_input)
        print("---OUTPUT-----")
        print(test_output)
        print(" "*50+"-"*50)

        # Save model
        file=open(iteration_file,'a')
        file.write('iteration:'+str(iteration+1)+'\n')
        file_path='/home/santhosh/resumes_folder/keras/extract_summary_and_objective/resume_line_level/model/resume_line_level_iteration_'+str(iteration+1)+'.h5'
        model.save(file_path)
        file.close()
        iteration+=1
        
    except:
        file=open(iteration_file,'r')
        last_line=file.read().split('\n')[-2]
        print('file_data,',last_line)
        iteration=int(last_line.split(':')[1])

        #print(iteration)
        file.close()
        # load weights
        print('loading the weights')
        file_path='/home/santhosh/resumes_folder/keras/extract_summary_and_objective/resume_line_level/model/resume_line_level_iteration_'+str(iteration)+'.h5'
        model=load_model(file_path)
        # estimate accuracy on whole dataset using loaded weights
        scores = model.evaluate([encoder_input_data, decoder_input_data], decoder_target_data,verbose=0)
        print("%s: %.2f%%\n\n" % (model.metrics_names[1], scores[1]*100))
        print("Testing Samples\n"+"-"*50)
        print("-"*50)
        index=int(np.random.randint(len(input_resumes)/40*0.8))
        test_input=""
        test_output=""
        for i in range(40):
            encoded_input_sequence=(encoder_input_data[index]).reshape((1,max_encoder_seq_len,num_encoder_tokens))
            #print(encoded_input_sequence.shape,max_encoder_seq_len,num_encoder_tokens,max_encoder_seq_len*num_encoder_tokens)
            #print(encoded_input_sequence)
            #.reshape((1,max_encoder_seq_len,num_encoder_tokens))
            output_sequence=decode_sequence(encoded_input_sequence)
            test_input+=input_resumes[index]
            test_output+=output_sequence
            index+=1
        print("-"*50)
        print(test_input)
        print("---OUTPUT-----")
        print(test_output)
        print(" "*50+"-"*50)


file_data, iteration:15
loading the weights
acc: 10.91%


Testing Samples
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
academic credentials: 
 b.e (information technology) with first class from pune university. 
 diploma in compute technology with first class under m.s.b.t.e.. 
 ssc with first class from pune board 
technical experience summary: 
technology tools 
functional areas manual and automation testing 
software engineering selenium ide(1.0.7),bugzilla, jira, test-link, hp qc, 
tools/technologies web-cruiser , havij, tortoisesvn, jmeter, jenkins 
 
abhishek appanna kalagoudra 
7#b/4, nav annapurna arti bldg, annapurna nagar, mobile: 9820385126 
adharwadi, kalyan(w).maharashtra-421301. email:abhishek.kalagoudra@gmail.com 
objective: 
want to work and implement my knowledge in an it organization to accomplish 
organizational goals and upgrade my knowledge. 
summary: 
 m

1634/1634 [==============================] - 90s 55ms/step - loss: 0.1637 - acc: 0.1170 - val_loss: 0.8047 - val_acc: 0.0486


/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/keras/engine/network.py:888: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1_6/while/Exit_2:0' shape=(?, 128) dtype=float32>, <tf.Tensor 'lstm_1_6/while/Exit_3:0' shape=(?, 128) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


Epoch 2/50
1634/1634 [==============================] - 75s 46ms/step - loss: 0.1337 - acc: 0.1237 - val_loss: 0.7698 - val_acc: 0.0645
Epoch 3/50
1634/1634 [==============================] - 75s 46ms/step - loss: 0.1184 - acc: 0.1257 - val_loss: 0.7611 - val_acc: 0.0584
Epoch 4/50
1634/1634 [==============================] - 74s 45ms/step - loss: 0.1066 - acc: 0.1290 - val_loss: 0.7620 - val_acc: 0.0585
Epoch 5/50
1634/1634 [==============================] - 73s 45ms/step - loss: 0.0949 - acc: 0.1305 - val_loss: 0.7614 - val_acc: 0.0572
Epoch 6/50
1634/1634 [==============================] - 73s 45ms/step - loss: 0.0871 - acc: 0.1284 - val_loss: 0.7719 - val_acc: 0.0570
Epoch 7/50
1634/1634 [==============================] - 72s 44ms/step - loss: 0.0819 - acc: 0.1296 - val_loss: 0.7697 - val_acc: 0.0637
Epoch 8/50
1634/1634 [==============================] - 73s 45ms/step - loss: 0.0751 - acc: 0.1302 - val_loss: 0.7662 - val_acc: 0.0573
Epoch 9/50
1634/1634 [==========================